In [1]:
import warnings

import numpy as np
import numpy.linalg as la
from numpy import log
from scipy.special import digamma
from sklearn.neighbors import BallTree, KDTree
import sklearn

import sys
import os
import matplotlib.pyplot as plt
import pandas as pd

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
#%% Cargo los datos
from sklearn.preprocessing import StandardScaler

# Functions

In [2]:
# DISCRETE ESTIMATORS
def entropyd(sx, base=2):
    """ Discrete entropy estimator
        sx is a list of samples
    """
    unique, count = np.unique(sx, return_counts=True, axis=0)
    # Convert to float as otherwise integer division results in all 0 for proba.
    proba = count.astype(float) / len(sx)
    # Avoid 0 division; remove probabilities == 0.0 (removing them does not change the entropy estimate as 0 * log(1/0) = 0.
    proba = proba[proba > 0.0]
    return np.sum(proba * np.log(1. / proba)) / log(base)

def centropyd(x, y, base=2):
    """ The classic K-L k-nearest neighbor continuous entropy estimator for the
        entropy of X conditioned on Y.
    """
    xy = np.c_[x, y]
    return entropyd(xy, base) - entropyd(y, base)

def midd(x, y, base=2):
    """ Discrete mutual information estimator
        Given a list of samples which can be any hashable object
    """
    assert len(x) == len(y), "Arrays should have same length"
    return entropyd(x, base) - centropyd(x, y, base)


def cmidd(x, y, z, base=2):
    """ Discrete mutual information estimator
        Given a list of samples which can be any hashable object
    """
    assert len(x) == len(y) == len(z), "Arrays should have same length"
    xz = np.c_[x, z]
    yz = np.c_[y, z]
    xyz = np.c_[x, y, z]
    return entropyd(xz, base) + entropyd(yz, base) - entropyd(xyz, base) - entropyd(z, base)


def entropy(x, k=3, base=2):
    """ The classic K-L k-nearest neighbor continuous entropy estimator
        x should be a list of vectors, e.g. x = [[1.3], [3.7], [5.1], [2.4]]
        if x is a one-dimensional scalar and we have four samples
    """
    assert k <= len(x) - 1, "Set k smaller than num. samples - 1"
    x = np.asarray(x)
    n_elements, n_features = x.shape
    x = add_noise(x)
    tree = build_tree(x)
    nn = query_neighbors(tree, x, k)
    const = digamma(n_elements) - digamma(k) + n_features * log(2)
    return (const + n_features * np.log(nn).mean()) / log(base)


def centropy(x, y, k=3, base=2):
    """ 
    The classic K-L k-nearest neighbor continuous entropy estimator for the
    entropy of X conditioned on Y.
    """
    xy = np.c_[x, y]
    entropy_union_xy = entropy(xy, k=k, base=base)
    entropy_y = entropy(y, k=k, base=base)
    return entropy_union_xy - entropy_y


def tc(xs, k=3, base=2):
    xs_columns = np.expand_dims(xs, axis=0).T
    entropy_features = [entropy(col, k=k, base=base) for col in xs_columns]
    return np.sum(entropy_features) - entropy(xs, k, base)


def ctc(xs, y, k=3, base=2):
    xs_columns = np.expand_dims(xs, axis=0).T
    centropy_features = [centropy(col, y, k=k, base=base)
                         for col in xs_columns]
    return np.sum(centropy_features) - centropy(xs, y, k, base)


def corex(xs, ys, k=3, base=2):
    xs_columns = np.expand_dims(xs, axis=0).T
    cmi_features = [mi(col, ys, k=k, base=base) for col in xs_columns]
    return np.sum(cmi_features) - mi(xs, ys, k=k, base=base)


def mi(x, y, z=None, k=3, base=2, alpha=0):
    """ 
    Mutual information of x and y (conditioned on z if z is not None)
    x, y should be a list of vectors, e.g. x = [[1.3], [3.7], [5.1], [2.4]]
    if x is a one-dimensional scalar and we have four samples
    """
    assert len(x) == len(y), "Arrays should have same length"
    assert k <= len(x) - 1, "Set k smaller than num. samples - 1"
    x, y = np.asarray(x), np.asarray(y)
    x, y = x.reshape(x.shape[0], -1), y.reshape(y.shape[0], -1)
    x = add_noise(x)
    y = add_noise(y)
    points = [x, y]
    if z is not None:
        z = np.asarray(z)
        z = z.reshape(z.shape[0], -1)
        points.append(z)
    points = np.hstack(points)
    # Find nearest neighbors in joint space, p=inf means max-norm
    tree = build_tree(points)
    dvec = query_neighbors(tree, points, k)
    if z is None:
        a, b, c, d = avgdigamma(x, dvec), avgdigamma(
            y, dvec), digamma(k), digamma(len(x))
        if alpha > 0:
            d += lnc_correction(tree, points, k, alpha)
    else:
        xz = np.c_[x, z]
        yz = np.c_[y, z]
        a, b, c, d = avgdigamma(xz, dvec), avgdigamma(
            yz, dvec), avgdigamma(z, dvec), digamma(k)
    return (-a - b + c + d) / log(base)


def cmi(x, y, z, k=3, base=2):
    """ Mutual information of x and y, conditioned on z
        Legacy function. Use mi(x, y, z) directly.
    """
    return mi(x, y, z=z, k=k, base=base)



def lnc_correction(tree, points, k, alpha):
    e = 0
    n_sample = points.shape[0]
    for point in points:
        # Find k-nearest neighbors in joint space, p=inf means max norm
        knn = tree.query(point[None, :], k=k+1, return_distance=False)[0]
        knn_points = points[knn]
        # Substract mean of k-nearest neighbor points
        knn_points = knn_points - knn_points[0]
        # Calculate covariance matrix of k-nearest neighbor points, obtain eigen vectors
        covr = knn_points.T @ knn_points / k
        _, v = la.eig(covr)
        # Calculate PCA-bounding box using eigen vectors
        V_rect = np.log(np.abs(knn_points @ v).max(axis=0)).sum()
        # Calculate the volume of original box
        log_knn_dist = np.log(np.abs(knn_points).max(axis=0)).sum()

        # Perform local non-uniformity checking and update correction term
        if V_rect < log_knn_dist + np.log(alpha):
            e += (log_knn_dist - V_rect) / n_sample
    return e



def tcd(xs, base=2):
    xs_columns = np.expand_dims(xs, axis=0).T
    entropy_features = [entropyd(col, base=base) for col in xs_columns]
    return np.sum(entropy_features) - entropyd(xs, base)


def ctcd(xs, y, base=2):
    xs_columns = np.expand_dims(xs, axis=0).T
    centropy_features = [centropyd(col, y, base=base) for col in xs_columns]
    return np.sum(centropy_features) - centropyd(xs, y, base)


def corexd(xs, ys, base=2):
    xs_columns = np.expand_dims(xs, axis=0).T
    cmi_features = [midd(col, ys, base=base) for col in xs_columns]
    return np.sum(cmi_features) - midd(xs, ys, base)


# MIXED ESTIMATORS
def micd(x, y, k=3, base=2, warning=True):
    """ If x is continuous and y is discrete, compute mutual information
    """
    assert len(x) == len(y), "Arrays should have same length"
    entropy_x = entropy(x, k, base)

    y_unique, y_count = np.unique(y, return_counts=True, axis=0)
    y_proba = y_count / len(y)

    entropy_x_given_y = 0.
    for yval, py in zip(y_unique, y_proba):
        x_given_y = x[(y == yval).all(axis=1)]
        if k <= len(x_given_y) - 1:
            entropy_x_given_y += py * entropy(x_given_y, k, base)
        else:
            if warning:
                warnings.warn("Warning, after conditioning, on y={yval} insufficient data. "
                              "Assuming maximal entropy in this case.".format(yval=yval))
            entropy_x_given_y += py * entropy_x
    return abs(entropy_x - entropy_x_given_y)  # units already applied


def midc(x, y, k=3, base=2, warning=True):
    return micd(y, x, k, base, warning)


def centropycd(x, y, k=3, base=2, warning=True):
    return entropy(x, base) - micd(x, y, k, base, warning)


def centropydc(x, y, k=3, base=2, warning=True):
    return centropycd(y, x, k=k, base=base, warning=warning)


def ctcdc(xs, y, k=3, base=2, warning=True):
    xs_columns = np.expand_dims(xs, axis=0).T
    centropy_features = [centropydc(
        col, y, k=k, base=base, warning=warning) for col in xs_columns]
    return np.sum(centropy_features) - centropydc(xs, y, k, base, warning)


def ctccd(xs, y, k=3, base=2, warning=True):
    return ctcdc(y, xs, k=k, base=base, warning=warning)


def corexcd(xs, ys, k=3, base=2, warning=True):
    return corexdc(ys, xs, k=k, base=base, warning=warning)


def corexdc(xs, ys, k=3, base=2, warning=True):
    return tcd(xs, base) - ctcdc(xs, ys, k, base, warning)


# UTILITY FUNCTIONS

def add_noise(x, intens=1e-10):
    # small noise to break degeneracy, see doc.
    return x + intens * np.random.random_sample(x.shape)


def query_neighbors(tree, x, k):
    return tree.query(x, k=k + 1)[0][:, k]


def count_neighbors(tree, x, r):
    return tree.query_radius(x, r, count_only=True)


def avgdigamma(points, dvec):
    # This part finds number of neighbors in some radius in the marginal space
    # returns expectation value of <psi(nx)>
    tree = build_tree(points)
    dvec = dvec - 1e-15
    num_points = count_neighbors(tree, points, dvec)
    return np.mean(digamma(num_points))


def build_tree(points):
    if points.shape[1] >= 20:
        return BallTree(points, metric='chebyshev')
    return KDTree(points, metric='chebyshev')




In [3]:
def firstMI(X, y, nTimeSteps, temporaryKeys, base=2):
    maxMI = 0
    indexMIMax = 0
    for k in range(len(temporaryKeys)):
        keys = [temporaryKeys[k] + str(s) for s in np.arange(0, nTimeSteps, 1).tolist()]
        miValue = np.abs(entropyd(y.values, base) - centropyd(y.values, X[keys].values, base))
        if miValue > maxMI:
            maxMI = miValue
            indexMIMax = k
    #Elimino la primera variable seleccionada de X y la añado a z
    keys = [temporaryKeys[indexMIMax] + str(s) for s in np.arange(0, nTimeSteps, 1).tolist()]
    z = X[keys].values
    X = X.drop(columns=keys)
    keyToReturn = temporaryKeys[indexMIMax]
    temporaryKeys.remove(keyToReturn)
    return X, z, keyToReturn, temporaryKeys, maxMI

def myCondMI(X, y, z, nTimeSteps, temporaryKeys, base=2):
    maxMI = 0
    indexMIMax = 0
    for k in range(len(temporaryKeys)):
        keys = [temporaryKeys[k] + str(s) for s in np.arange(0, nTimeSteps, 1).tolist()]
        miValue = np.abs(cmidd(y.values, X[keys].values, z))
        if miValue > maxMI:
            maxMI = miValue
            indexMIMax = k
    #Elimino la primera variable seleccionada de X y la añado a z
    keys = [temporaryKeys[indexMIMax] + str(s) for s in np.arange(0, nTimeSteps, 1).tolist()]
    z = np.append(z, X[keys].values, axis=1)
    X = X.drop(columns=keys)
    keyToReturn = temporaryKeys[indexMIMax]
    temporaryKeys.remove(keyToReturn)
    return X, z, keyToReturn, temporaryKeys, maxMI

# Code - Dynamic CMI

In [4]:
i = 0
n = 4

X_train = np.load("../splits_14_days/notbalanced/split_" + str(i) +
                                              "/X_train_tensor_" + str(n)+ ".npy")
y_train = pd.read_csv("../splits_14_days/notbalanced/split_" + str(i) +
                      "/y_train_" + str(n)+ ".csv",
                     index_col=0)

X_val = np.load("../splits_14_days/notbalanced/split_" + str(i) +
                                              "/X_val_tensor_" + str(n)+ ".npy")
y_val = pd.read_csv("../splits_14_days/notbalanced/split_" + str(i) +
                    "/y_val_" + str(n)+ ".csv",
                   index_col=0)


temporaryKeys = ['AMG', 'ATF', 'CAR', 'CF1', 'CF2', 'CF3',
       'CF4', 'Falta', 'GCC', 'GLI', 'LIN', 'LIP', 'MAC', 'MON', 'NTI', 'OTR',
       'OXA', 'PAP', 'PEN', 'POL', 'QUI', 'SUL', 'TTC', 'pc_acinet',
       'pc_enterob', 'pc_enteroc', 'pc_pseud', 'pc_staph', 'pc_stenot',
       'pc_no_germ', 'isVM', 'numberOfPatients', 'numberOfPatientsMR',
       'neighbor_AMG', 'neighbor_ATF', 'neighbor_CAR', 'neighbor_CF1',
       'neighbor_CF2', 'neighbor_CF3', 'neighbor_CF4', 'neighbor_Falta',
       'neighbor_GCC', 'neighbor_GLI', 'neighbor_LIN', 'neighbor_LIP',
       'neighbor_MAC', 'neighbor_MON', 'neighbor_NTI', 'neighbor_OTR',
       'neighbor_OXA', 'neighbor_PAP', 'neighbor_PEN', 'neighbor_POL',
       'neighbor_QUI', 'neighbor_SUL', 'neighbor_TTC']

data_train = []
data_val = []

# Transform the .npy into a .csv breaking the temporal dimensions
for patient in range(X_train.shape[0]): 
    row_data = {}  
    for feature_idx, feature_name in enumerate(temporaryKeys): 
        for time_step in range(X_train.shape[1]):  
            # Create the column 'feature_name' + time_step: AMG0, AMG1, etc.
            column_name = f"{feature_name}{time_step}"
            row_data[column_name] = X_train[patient, time_step, feature_idx]  
    data_train.append(row_data) 


for patient in range(X_val.shape[0]): 
    row_data = {}  
    for feature_idx, feature_name in enumerate(temporaryKeys): 
        for time_step in range(X_val.shape[1]):  
            # Create the column 'feature_name' + time_step
            column_name = f"{feature_name}{time_step}"
            row_data[column_name] = X_val[patient, time_step, feature_idx]  
    data_val.append(row_data)  

df_train = pd.DataFrame(data_train)
df_val = pd.DataFrame(data_val)

In [5]:
X = pd.concat([df_train, df_val], axis=0)
y = pd.concat([y_train, y_val], axis=0)

In [6]:
NTimeSteps = 14
NFeatures = 56

temporaryKeys = ['AMG', 'ATF', 'CAR', 'CF1', 'CF2', 'CF3',
       'CF4', 'Falta', 'GCC', 'GLI', 'LIN', 'LIP', 'MAC', 'MON', 'NTI', 'OTR',
       'OXA', 'PAP', 'PEN', 'POL', 'QUI', 'SUL', 'TTC', 'pc_acinet',
       'pc_enterob', 'pc_enteroc', 'pc_pseud', 'pc_staph', 'pc_stenot',
       'pc_no_germ', 'isVM', 'numberOfPatients', 'numberOfPatientsMR',
       'neighbor_AMG', 'neighbor_ATF', 'neighbor_CAR', 'neighbor_CF1',
       'neighbor_CF2', 'neighbor_CF3', 'neighbor_CF4', 'neighbor_Falta',
       'neighbor_GCC', 'neighbor_GLI', 'neighbor_LIN', 'neighbor_LIP',
       'neighbor_MAC', 'neighbor_MON', 'neighbor_NTI', 'neighbor_OTR',
       'neighbor_OXA', 'neighbor_PAP', 'neighbor_PEN', 'neighbor_POL',
       'neighbor_QUI', 'neighbor_SUL', 'neighbor_TTC']


NSubFeatures = np.zeros(X.shape[1])

indexesSelected = []
MIvalues = []
for j in range(NFeatures):
    if j == 0:
        X, z, featureSelected, temporaryKeys, maxMI = firstMI(X, y, NTimeSteps, temporaryKeys)
        indexesSelected.append(featureSelected)
        MIvalues.append(maxMI)
        print("Primera feature seleccionada:", featureSelected)
        print(X.shape)
    else:
        X, z, featureSelected, temporaryKeys, maxMI = myCondMI(X, y, z, NTimeSteps, temporaryKeys)
        indexesSelected.append(featureSelected)
        MIvalues.append(maxMI)
        print("Siguiente feature seleccionada:", featureSelected)
        print(X.shape)
        print(len(temporaryKeys))
results = pd.DataFrame(data=np.array([np.array(indexesSelected), np.array(MIvalues)]).T, columns=["Keys", "MIValues"])

Primera feature seleccionada: neighbor_QUI
(2526, 770)
Siguiente feature seleccionada: neighbor_CAR
(2526, 756)
54
Siguiente feature seleccionada: neighbor_PAP
(2526, 742)
53
Siguiente feature seleccionada: neighbor_OXA
(2526, 728)
52
Siguiente feature seleccionada: numberOfPatients
(2526, 714)
51
Siguiente feature seleccionada: neighbor_CF3
(2526, 700)
50
Siguiente feature seleccionada: neighbor_PEN
(2526, 686)
49
Siguiente feature seleccionada: neighbor_GLI
(2526, 672)
48
Siguiente feature seleccionada: CAR
(2526, 658)
47
Siguiente feature seleccionada: ATF
(2526, 644)
46
Siguiente feature seleccionada: PAP
(2526, 630)
45
Siguiente feature seleccionada: neighbor_NTI
(2526, 616)
44
Siguiente feature seleccionada: AMG
(2526, 602)
43
Siguiente feature seleccionada: CF1
(2526, 588)
42
Siguiente feature seleccionada: CF2
(2526, 574)
41
Siguiente feature seleccionada: CF3
(2526, 560)
40
Siguiente feature seleccionada: CF4
(2526, 546)
39
Siguiente feature seleccionada: Falta
(2526, 532)
38


In [7]:
results.to_csv("results_MI_COND_dynamic.csv")